In [56]:
#NEED TO RESTART KERNEL TO LOAD CHANGES TO IMPORTS :(

import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import data_loader
import data_cache
from energy_channels import EnergyChannel
import plot_tools
import datetime
import matplotlib.pyplot as plt
import font_sizes
import numpy as np
import field_models
import psd_tool
import scipy

import importlib
importlib.reload(plot_tools)
importlib.reload(field_models)
importlib.reload(psd_tool)
importlib.reload(data_loader)
importlib.reload(data_cache)

%matplotlib qt

#NEED TO RESTART KERNEL TO LOAD CHANGES TO IMPORTS :(

In [57]:
start = datetime.datetime(year = 2015, month = 1, day = 1)
end = datetime.datetime(year = 2015, month = 2, day = 1)
sat = "A"

In [6]:
REPT = data_loader.load_compressed_rept_data(satellite=sat, start=start, end=end)

fig, axs = plt.subplots(3, 1, sharex=True)
fig.set_size_inches(16, 9)
fig.set_layout_engine("constrained")

E1 = EnergyChannel.MeV_1_8
E2 = EnergyChannel.MeV_2_1
E3 = EnergyChannel.MeV_2_6

plot_tools.plot_l_vs_time_log_colors(x = REPT["EPOCH"], y = REPT["L"], c = REPT["FESA"][:, E1.value], axis=axs[0], cbar_label="($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)\n")
plot_tools.plot_l_vs_time_log_colors(x = REPT["EPOCH"], y = REPT["L"], c = REPT["FESA"][:, E2.value], axis=axs[1], cbar_label="($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)\n")
plot_tools.plot_l_vs_time_log_colors(x = REPT["EPOCH"], y = REPT["L"], c = REPT["FESA"][:, E3.value], axis=axs[2], cbar_label="($cm^{-2}s^{-1}sr^{-1}MeV^{-1}$)\n")
    
axs[0].set_xlim(start, end)
axs[1].set_xlim(start, end)
axs[2].set_xlim(start, end)

axs[0].set_ylim(3, 6)
axs[1].set_ylim(3, 6)
axs[2].set_ylim(3, 6)


axs[0].set_title(f"{REPT["ENERGIES"][E1.value]} MeV", fontsize = 10)
axs[1].set_title(f"{REPT["ENERGIES"][E2.value]} MeV", fontsize = 10)
axs[2].set_title(f"{REPT["ENERGIES"][E3.value]} MeV", fontsize = 10)



fig.suptitle(f"RBSP-{sat}, Spin Averaged Electron Flux", fontsize = 14)
plt.show()

Loading REPT data between: 2015-06-01 00:00:00 and 2015-07-01 00:00:00.
Loading : REPT_201506_A.npz
Loading : REPT_201507_A.npz


In [58]:
field_model = field_models.model.TS04D

mu_vals = [500, 1000, 3000]
chosen_k = 0.07

In [59]:
psd_refs = data_loader.load_psd(satellite = sat, field_model = field_model, start = start, end = end)

2015-01-01 00:00:00
Loading : PSD_201501_A_TS04D.npz
2015-02-01 00:00:00
Loading : PSD_201502_A_TS04D.npz


In [60]:
for chosen_mu in mu_vals:
    
    print(f"Selecting PSD for A at mu = {chosen_mu}, k = {chosen_k}")
    
    selected_psd_refs = psd_tool.select_mu_and_k_from_psd(refs=psd_refs, chosen_mu=chosen_mu, chosen_k=chosen_k)
    
    data_cache.cache_psd_at_selected_mu_and_k(selected_psd_refs, satellite = sat, start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)


Selecting PSD for A at mu = 500, k = 0.07
Time taken for loop: 5.132992506027222
Selecting PSD for A at mu = 1000, k = 0.07
Time taken for loop: 5.165690660476685
Selecting PSD for A at mu = 3000, k = 0.07
Time taken for loop: 4.999297618865967


In [61]:
Lstar_min = 3.0
Lstar_max = 6.0

fig, axs = plt.subplots(len(mu_vals), 1, sharex=True)
fig.set_size_inches(16, 9)
fig.set_layout_engine("constrained")

for k, chosen_mu in enumerate(mu_vals):
    
    selected_psd_refs = data_cache.load_psd_cache(satellite = sat, start = start, end = end, mu = chosen_mu, k = chosen_k, field_model = field_model)
    
    epoch = selected_psd_refs["EPOCH"]
    Lstar = selected_psd_refs["L_STAR"]
    PSD = selected_psd_refs["PSD"]
    
    outer_belt = (3 < Lstar) & (Lstar < 7)
    
    plot_tools.plot_l_vs_time_log_colors(x = epoch[outer_belt], y = Lstar[outer_belt], c = PSD[outer_belt], cbar_label = "PSD (c/Mev/G)", axis = axs[k])
    axs[k].set_title(rf"RBSP-{sat}, K = {chosen_k}, ($R_E$$G^{{1/2}}$), $\mu$ = {chosen_mu} (MeV / G), {field_model.name}")
    axs[k].set_ylabel("L*")
    axs[k].set_ylim(Lstar_min, Lstar_max)

axs[0].set_xlim(start, end)

plt.show()    